# Normalizing energy consumption of buildings to be able to compare them to each other

- Created by Clayton Miller - clayton@nus.edu.sg - miller.clayton@gmail.com

Buildings come in all shapes, sizes and uses. It is difficult to compare the energy consumption of two buildings if they are not similar in these ways as it wouldn't be an *apples to apples* comparison. In order to compare buildings of different uses, sizes, and climates -- we have to do a process called `normalization`.



## The question -- how can we compare the energy consumption of buildings that are different sizes, from different climates, etc.?

To get started with this task, we can use the the BDG data set from several buildings. First we'll load the libraries and data once again.


In [0]:
import pandas as pd
from google.colab import drive
import os

In [0]:
%matplotlib inline

In [0]:
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/EDX Data Science for Construction, Architecture and Engineering/Week 3 - Construction - Pandas Fundamentals/meter_data/")

In [0]:
buildingname = "Office_Abbey"

In [0]:
rawdata = pd.read_csv(buildingname+".csv", parse_dates=True, index_col='timestamp')

In [0]:
rawdata.info()

In [0]:
rawdata.head()

In [0]:
rawdata.plot(figsize=(20,8))

# Normalization based on floor area

One of the most simple ways of normalization is to simply divide energy consumption by the floor area to create the Energy Use Intensity (EUI). This metric takes energy and simply divides by the floor area (in ft2 or m2)

In [0]:
meta = pd.read_csv("/content/gdrive/My Drive/EDX Data Science for the Built Environment/3 - Construction - Pandas Fundamentals/all_buildings_meta_data.csv", index_col="uid")

In [0]:
meta.head()

## Extract meta data from another file

You can see in the meta data file that there are various attributes that we can use to normalize the building consumption -- including `sqm` or size of the building

In [0]:
meta.loc[buildingname]

In [0]:
meta.loc[buildingname]["sqm"]

Now we normalize using area by using a vector operation and resampling to monthly data.

In [0]:
rawdata.head()

In [0]:
rawdata_normalized = rawdata/meta.loc[buildingname]["sqm"]

In [0]:
rawdata_normalized.head()

In [0]:
rawdata_normalized_monthly = rawdata_normalized.resample("M").sum()

In [0]:
rawdata_normalized_monthly.plot(kind="bar", figsize=(10,4), title='Energy Consumption per Square Meter Floor Area')

# Automation of the process of analysis on multiple buildings

Normalization is really about the ability to compare a diverse set of buildings

Now we load 6 buildings to extract and normalize the data. Since the normalization process is most useful on a set of buildings, we will load six of the BDG buildings as an example to showcase the comparison.

In [0]:
buildingnamelist = ["Office_Abbey",
"Office_Pam",
"Office_Penny",
"UnivLab_Allison",
"UnivLab_Audra",
"UnivLab_Ciel"]

In [0]:
annual_data_list = []
annual_data_list_normalized = []

In [0]:
for buildingname in buildingnamelist:
    print("Getting data from: "+buildingname)
    
    rawdata = pd.read_csv(buildingname+".csv", parse_dates=True, index_col='timestamp')
    floor_area = meta.loc[buildingname]["sqm"]
    
    annual = rawdata.sum()

    normalized_data = rawdata/floor_area
    annual_normalized = normalized_data.sum()
    
    annual_data_list_normalized.append(annual_normalized)
    annual_data_list.append(annual) 

In [0]:
totaldata = pd.concat(annual_data_list)
totaldata_normalized = pd.concat(annual_data_list_normalized)

In [0]:
totaldata

In [0]:
totaldata_normalized

## Unnormalized energy consumption

As you can see from the annual totals of energy consumption, the Lab named *Ciel* uses a lot of energy as compared to the other buildings!

In [0]:
totaldata.plot(kind='bar',figsize=(10,5))

## Normalized Energy Consumption

However, when normalized using floor area, *Ciel* is not the highest consumer

In [0]:
totaldata_normalized.plot(kind='bar',figsize=(10,5))